In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import csv
from scipy.optimize import minimize_scalar

# Open files 

In [ ]:
with open("observations", "r") as f:
    obs_df = pd.read_csv(f, index_col=0)

f.close()

In [ ]:
with open("inputs", "r") as f:
    inputs_df = pd.read_csv(f, index_col=0)

f.close()

In [ ]:
with open("model_variants", "r") as f:
    model_variants_df = pd.read_csv(f, index_col=0)

f.close()

In [ ]:
with open("outliers", "r") as f:
    obs_df2 = pd.read_csv(f, index_col=0)

f.close()

In [ ]:
days = [str(n).zfill(2) for n in range(1, 15)]
times = ["09_20_00", "12_20_00"]

# Since the predictions take up so much space, they are separated by day
prediction_sets = ["predictions_07_" + day + "_17_" + time for day in days for time in times]

## Step 1: Get variance terms for strict bounds and history matching methods

In [ ]:
idxSet=(obs_df2['missing']) | (obs_df2['outlier'])

In [ ]:
def get_all_squares(idxSet=list((obs_df2['missing']) | (obs_df2['outlier'])), method='sb'):
    """
    y = observed AOD
    zs = emulated AODs (vector)
    
    e = estimated instrument error standard deviation
    ss = standard deviation of AOD emulation


    Arguments

    idxSet : Set of row indices which are to be excluded from analysis
    method : Which method to use for estimating uncertainties, either 'sb' (strict bounds, our method) or 'hm' (history
        matching, based on Johnson et al. (2020))

    
    Value
    
    Tuple : "Distances" (differences in response) and "variances" (terms needed to normalize the distances)
    """
    allDistances = []
    allVariances = []
    
    my_obs_df = obs_df.copy()
    my_obs_df.loc[idxSet, ["meanResponse", "sdResponse"]] = [float("nan"), float("nan")]

    for time, prediction_set in zip(np.unique(my_obs_df.time), prediction_sets):
        
        my_obs_df_this_time = my_obs_df[my_obs_df.time==time].reset_index(drop=True)
        num_pixels = len(my_obs_df_this_time.index)
        
        with open(prediction_set, "r") as f:
            my_predict_df_this_time = pd.read_csv(f, index_col=0)
        
        my_predict_dfs = [
            my_predict_df_this_time.iloc[k*5000:(k+1)*5000, :].reset_index(drop=True) 
            for k in range(num_pixels)
        ]

        for row in range(num_pixels):

            y = my_obs_df_this_time.loc[row, 'meanResponse']
            if method=='sb':
                e = my_obs_df_this_time.loc[row, 'sdResponse']**2
            elif method=='hm':
                # Per Johnson et al. (2020), instrument uncertainty is 10%, spatial co-location uncertainty is 20%, and
                # temporal sampling uncertainty is 10% of the measured value.
                e = (0.1+0.2+0.1)*y

            zs = my_predict_dfs[row]['meanResponse']
            ss = my_predict_dfs[row]['sdResponse']**2

            if ~np.isnan(y) and y != 0:
                distances = list(y - zs)
                variances = list(e + ss)
            else:
                distances = [float('nan')]*len(zs)
                variances = [float('nan')]*len(zs)

            allDistances.append(pd.DataFrame(distances).transpose())
            allVariances.append(pd.DataFrame(variances).transpose())

    return (
        pd.concat(allDistances, axis=0).reset_index(drop=True),
        pd.concat(allVariances, axis=0).reset_index(drop=True)
    )

### For strict bounds

In [ ]:
my_distances, my_variances = get_all_squares()

In [ ]:
my_variances

In [ ]:
with open('distances', 'w') as f:
    my_distances.to_csv(f)
f.close()

In [ ]:
with open('variances', 'w') as f:
    my_variances.to_csv(f)
f.close()

In [ ]:
sums_squares = np.power(my_distances, 2).sum(axis=0)
sums_squares

### For history matching

In [ ]:
my_hm_distances, my_hm_variances = get_all_squares(method='hm')

In [ ]:
my_hm_variances

In [ ]:
with open('hm_variances', 'w') as f:
    my_hm_variances.to_csv(f)
f.close()

## Step 2: Compute MLE for outstanding variance term

In [ ]:
def l(d, u):
    # Log likelihood, to be maximized
    term1 = np.nansum(np.log(my_variances.iloc[:, u] + d))
    term2 = np.nansum(np.power(my_distances.iloc[:, u], 2) / (my_variances.iloc[:, u] + d))
    return 0.5 * (term1 + term2)

In [ ]:
max_l_for_us = []

for u in range(5000):
    res = minimize_scalar(l, args=(u))
    max_l_for_us.append(-res.fun)

In [ ]:
u_mle = max_l_for_us.index(max(max_l_for_us))

In [ ]:
u_mle

In [ ]:
additional_variance = minimize_scalar(l, args=(u_mle)).x

In [ ]:
additional_variance

In [ ]:
l(additional_variance, u_mle)

In [ ]:
with open('mle', 'w') as f:
    write = csv.writer(f)
    write.writerow([u_mle, additional_variance])
f.close()